In [2]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
import numpy as np


In [3]:
# Create the dataframe from the csv file
players = pd.read_csv("Seasons_Stats.csv")

# Get the head
players.head()
list(players.columns.values)
players.dtypes

Unnamed: 0      int64
Year          float64
Player         object
Pos            object
Age           float64
Tm             object
G             float64
GS            float64
MP            float64
PER           float64
TS%           float64
3PAr          float64
FTr           float64
ORB%          float64
DRB%          float64
TRB%          float64
AST%          float64
STL%          float64
BLK%          float64
TOV%          float64
USG%          float64
blanl         float64
OWS           float64
DWS           float64
WS            float64
WS/48         float64
blank2        float64
OBPM          float64
DBPM          float64
BPM           float64
VORP          float64
FG            float64
FGA           float64
FG%           float64
3P            float64
3PA           float64
3P%           float64
2P            float64
2PA           float64
2P%           float64
eFG%          float64
FT            float64
FTA           float64
FT%           float64
ORB           float64
DRB       

In [8]:
# Drop unneeded columns, keep identifier columns
# Keep PER: "Player Efficiency Rating"
# Keep WS: "Win Shares"
# Keep BPM: "Box Plus-Minus"
# Keep VORP: "Value Over Replacement Player"
adv_players = players.filter(['Year','Player', 'G', 'MP', 'Pos','Tm', 'PER', 'WS', 'BPM', 'VORP'], axis=1)
adv_players['MP/G'] = adv_players['MP'] / adv_players['G']

# Tidy the data to only include season stats from 2007 - present
adv_players = adv_players.drop(adv_players[adv_players.Year < 2007].index)
adv_players = adv_players.drop(adv_players[adv_players.MP < 1000].index)
adv_players = adv_players[np.isfinite(adv_players['Year'])]

adv_players

,Year,Player,G,MP,Pos,Tm,PER,WS,BPM,VORP,MP/G
18226,2007.0,Shareef Abdur-Rahim,80.0,2015.0,C,SAC,13.1,2.8,-2.1,-0.1,25.187500
18229,2007.0,LaMarcus Aldridge,63.0,1392.0,C,POR,17.1,3.4,-0.9,0.4,22.095238
18231,2007.0,Ray Allen,55.0,2219.0,SG,SEA,21.6,6.3,3.4,3.0,40.345455
18233,2007.0,Rafer Alston,82.0,3040.0,PG,HOU,12.9,6.6,1.3,2.5,37.073171
18238,2007.0,Derek Anderson,50.0,1190.0,SF,CHA,13.7,2.9,1.0,0.9,23.800000
18239,2007.0,Carmelo Anthony,65.0,2486.0,SF,DEN,22.1,7.3,2.0,2.5,38.246154
18241,2007.0,Gilbert Arenas,74.0,2942.0,PG,WAS,24.0,10.8,5.0,5.2,39.756757
18242,2007.0,Trevor Ariza,57.0,1278.0,SG,ORL,16.2,3.4,1.5,1.1,22.421053
18243,2007.0,Darrell Armstrong,81.0,1275.0,PG,IND,15.1,3.2,1.1,1.0,15.740741
18245,2007.0,Carlos Arroyo,72.0,1304.0,PG,ORL,14.4,2.5,-2.0,0.0,18.111111


In [9]:
groups = adv_players.groupby('Year')

group07 = groups.get_group(2007.0)
group08 = groups.get_group(2008.0)
group09 = groups.get_group(2009.0)
group10 = groups.get_group(2010.0)
group11 = groups.get_group(2011.0)
group12 = groups.get_group(2012.0)
group13 = groups.get_group(2013.0)
group14 = groups.get_group(2014.0)
group15 = groups.get_group(2015.0)
group16 = groups.get_group(2016.0)
group17 = groups.get_group(2017.0)

group17

,Year,Player,G,MP,Pos,Tm,PER,WS,BPM,VORP,MP/G
24096,2017.0,Alex Abrines,68.0,1055.0,SG,OKC,10.1,2.1,-2.5,-0.1,15.514706
24100,2017.0,Steven Adams,80.0,2389.0,C,OKC,16.5,6.4,0.5,1.5,29.862500
24101,2017.0,Arron Afflalo,61.0,1580.0,SG,SAC,9.0,1.4,-3.5,-0.6,25.901639
24104,2017.0,LaMarcus Aldridge,72.0,2335.0,PF,SAS,18.6,7.3,1.1,1.8,32.430556
24106,2017.0,Tony Allen,71.0,1914.0,SG,MEM,13.3,3.1,0.7,1.3,26.957746
24107,2017.0,Al-Farouq Aminu,61.0,1773.0,SF,POR,11.3,1.9,-1.1,0.4,29.065574
24110,2017.0,Justin Anderson,75.0,1228.0,SF,TOT,13.9,1.9,-1.6,0.1,16.373333
24113,2017.0,Kyle Anderson,72.0,1020.0,SG,SAS,12.5,2.7,1.6,0.9,14.166667
24114,2017.0,Ryan Anderson,72.0,2116.0,PF,HOU,13.5,5.2,-0.8,0.6,29.388889
24115,2017.0,Giannis Antetokounmpo,80.0,2845.0,SF,MIL,26.1,12.4,7.6,6.9,35.562500


In [6]:
PerGroup17 = group17.sort_values(by='VORP', ascending=False)
PerGroup17.head(n=25)

,Year,Player,G,MP,Pos,Tm,PER,WS,BPM,VORP
24654,2017.0,Russell Westbrook,81.0,2802.0,PG,OKC,30.6,13.1,15.6,12.4
24306,2017.0,James Harden,81.0,2947.0,PG,HOU,27.3,15.0,10.1,9.0
24365,2017.0,LeBron James,74.0,2794.0,SF,CLE,27.0,12.9,8.4,7.3
24115,2017.0,Giannis Antetokounmpo,80.0,2845.0,SF,MIL,26.1,12.4,7.6,6.9
24175,2017.0,Jimmy Butler,76.0,2809.0,SF,CHI,25.1,13.8,6.9,6.3
24414,2017.0,Kawhi Leonard,74.0,2474.0,SF,SAS,27.5,13.6,7.9,6.2
24216,2017.0,Stephen Curry,79.0,2638.0,PG,GSW,24.6,12.6,7.3,6.2
24625,2017.0,Karl-Anthony Towns,82.0,3030.0,C,MIN,26.0,12.7,5.0,5.4
24285,2017.0,Rudy Gobert,81.0,2744.0,C,UTA,23.3,14.3,5.8,5.4
24380,2017.0,Nikola Jokic,73.0,2038.0,C,DEN,26.4,9.7,8.3,5.3
